### 🧠 What is Query Decomposition?
Query decomposition is the process of taking a complex, multi-part question and breaking it into simpler, atomic sub-questions that can each be retrieved and answered individually.

#### ✅ Why Use Query Decomposition?

- Complex queries often involve multiple concepts

- LLMs or retrievers may miss parts of the original question

- It enables multi-hop reasoning (answering in steps)

- Allows parallelism (especially in multi-agent frameworks)

In [1]:
from langchain.chat_models import init_chat_model
from langchain.prompts import PromptTemplate
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.runnables import RunnableSequence

In [3]:
# Step 1: Load and embed the document
loader = TextLoader('langchain_crewai_dataset.txt')
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size = 300, chunk_overlap = 50)
chunks = splitter.split_documents(docs)

embeddings = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")
vectorestore = FAISS.from_documents(chunks, embeddings)
retriever = vectorestore.as_retriever(search_type = "mmr", search_kwargs = {"k":4, "lambda_mult":0.7})

e:\workspace\ultimate_rag_bootcamp\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

llm = init_chat_model("openai:gpt-4o-mini")

In [10]:
# Step 3: Query decomposition
decomposition_prompt = PromptTemplate.from_template("""
You are an AI assistant. Decompose the following complex question into 2 to 4 smaller sub-questions for better document retrieval.

Question: "{question}"

Sub-questions:
""")
decomposition_chain = decomposition_prompt | llm | StrOutputParser()

In [ ]:
query = "How does Langchain use memory and agents compared to CrewAI?"
decompostion_question = decomposition_chain.invoke({"question":query}) 

In [12]:
print(decompostion_question)

1. What is the memory architecture utilized by Langchain, and how does it function in its framework?
2. How does Langchain implement the use of agents, and what roles do they play in its system?
3. What memory features and functionalities does CrewAI offer, and how do they differ from those of Langchain?
4. How does the agent-based approach in CrewAI compare to that of Langchain in terms of capabilities and applications?


In [18]:
for q in decompostion_question.split("\n"):
    if q.strip():
        print([q.strip("-.1234567890. ")])

['What is the role of memory in Langchain, and how is it implemented in its architecture?']
['How does Langchain utilize agents, and what are the key functionalities of these agents?']
["What is the concept of memory in CrewAI, and how does it differ from Langchain's implementation?"]
['How does CrewAI utilize agents, and what are the main differences in agent functionalities compared to Langchain?']


In [13]:
# step4:  QA chain per sub-question
qa_prompt = PromptTemplate.from_template("""
Use the context below to answer the question.

Context:
{context}

Question: {input}
""")

qa_chain = create_stuff_documents_chain(llm, qa_prompt)

In [21]:
#Step5" Full RAG pipeline logic

def full_query_decomposition_rag_pipeline(user_query):
    #Decompose the query
    sub_qs_text = decomposition_chain.invoke({"question":user_query})
    sub_questions = [q.strip("-•1234567890. ").strip() for q in sub_qs_text.split("\n") if q.strip()]

    results = []
    for subq in sub_questions:
        docs = retriever.invoke(subq)
        result = qa_chain.invoke({"input":subq, "context":docs})
        results.append(f"Q: {subq} \n A:{result}")
    
    return "\n\n".join(results)

In [22]:
#Step6 Run
query = "How does LangChain use memory and agents compared to CrewAI?"
final_answer = full_query_decomposition_rag_pipeline(query)
print("✅ Final Answer:\n")
print(final_answer)


✅ Final Answer:

Q: What is the role of memory in LangChain and how is it implemented? 
 A:In LangChain, memory plays a crucial role in enabling the language model (LLM) to maintain context and continuity throughout conversations. It allows the model to keep track of previous conversation turns, ensuring that interactions feel coherent and contextually aware. This is particularly important in applications where the conversation may extend over several exchanges or where users expect personalized responses based on earlier interactions.

LangChain implements memory through modules like **ConversationBufferMemory** and **ConversationSummaryMemory**. 

1. **ConversationBufferMemory**: This module allows the LLM to keep track of the entire conversation history, storing previous exchanges in a buffer. This means that the LLM can reference past interactions directly as the conversation progresses.

2. **ConversationSummaryMemory**: In scenarios where conversations are lengthy and may exceed 